📘 Section 1: Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

📘 Section 2: Loading and Restructuring the Dataset

In [ ]:
# Load raw CSV data (no header)
csv_file = "Dados-SLA.csv"
raw_data = pd.read_csv(csv_file, header=None)

# Define number of clients and expected columns (1 ID + 11 responses)
num_clients = 119
num_factors = 12

# Reshape flat data into structured 2D array
reshaped_data = raw_data.values.reshape(num_clients, num_factors)

# Define column names
columns = [
    "Client ID", "Service Rate", "Completed Orders", "Speed", "Consistency",
    "Flexibility", "Failure Recovery", "Information", "Correct Invoices",
    "Conforming Products", "Correct Quantity", "Overall Satisfaction"
]

# Create a DataFrame with the structured data
data = pd.DataFrame(reshaped_data, columns=columns)